# Zadanie 9

In [79]:
import numpy as np
from math import ceil

In [71]:
def majority(neighbours):
    return int(sum(neighbours) > 1)


def GKL(initial_config, j, k, steps):
    diagram = [initial_config]
    n = len(initial_config)
    padded_config = np.pad(initial_config, (k, k), 'wrap')

    for _ in range(steps):
        neighbourhoods = []
        for idx in range(k, n + k):
            if padded_config[idx] == 0:
                neighbourhoods.append([padded_config[idx], padded_config[idx - j], padded_config[idx - k]])
            else:
                neighbourhoods.append([padded_config[idx], padded_config[idx + j], padded_config[idx + k]])

        result = [majority(neighbourhood) for neighbourhood in neighbourhoods]
        diagram.append(result)
        padded_config = np.pad(result, (k, k), 'wrap')

    return diagram


def DCP(diagram):
    initial_density = np.mean(diagram[0])  # Calculate initial density
    final_density = np.mean(diagram[-1])  # Calculate final density

    if initial_density < 0.5:
        return final_density == 0
    return final_density == 1


In [77]:
def convert_configuration(configuration: bytes, N: int):
    binary_str = ''.join(f'{byte:08b}' for byte in configuration)
    return np.array([int(bit) for bit in binary_str[:N]])


def configuration_reader(file_name: str, N: int, negate: bool):
    bytes_per_configuration = ceil(N / 8)
    configurations = []

    with open(file_name, 'rb') as file:
        while True:
            bytes_read = file.read(bytes_per_configuration)
            if not bytes_read:
                break

            cfg = convert_configuration(bytes_read, N)
            configurations.append(cfg)

            if negate:
                configurations.append(1 - cfg)

    return np.stack(configurations)

In [73]:
GKL([0, 1, 1, 0, 0, 1, 0, 0], 1, 3, 3)

[[0, 1, 1, 0, 0, 1, 0, 0],
 [0, 1, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0]]

In [75]:
DCP(GKL([0, 1, 1, 0, 0, 1, 0, 0], 1, 3, 2))

False

In [80]:
configurations = configuration_reader("ALL_N21.bin", 21, True)

In [92]:
results = []

for config in configurations:
    results.append(DCP(GKL(config, 1, 3, 42)))

225.28679394721985


In [93]:
sum(results) / len(results)

0.90778934721666